## Introduction

An Ecuadorian family wants to live the American dream, for that they want to open a restaurant with traditional Latin American food. The main dish they want to offer is the "encebollado", with this they also want to offer sea food.

My idea for this project is to analyse which neighborhood would have better profitability their idea. With data they want to know which neignborhood they should approach.This idea can be scalable for many Latin American restaurants that want to be open in New York, specific in Queens


## Data

For this project I will use:

- Query the FourSqaure website for the neighboorhods of New York where lives Latin people.
- New York neighboorhods.
- Number of restaurants in the neighborhood

### Extracting Neighborhoods

In [1]:
!pip install geopy
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import folium 

     |████████████████████████████████| 102kB 931kB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


Let's create a dataframe with latitud and longitud of Nwe York City

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Now lets define the neighborhoods in Queens 

In [3]:
neighborhoods=neighborhoods[neighborhoods.Borough == 'Queens']
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
129,Queens,Astoria,40.768509,-73.915654
130,Queens,Woodside,40.746349,-73.901842
131,Queens,Jackson Heights,40.751981,-73.882821
132,Queens,Elmhurst,40.744049,-73.881656
133,Queens,Howard Beach,40.654225,-73.838138


In [4]:
address = 'Queens, New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
queens_latitude = location.latitude
queens_longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(queens_latitude, queens_longitude))

The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


A plot of the neighborhoods in Queens

In [5]:
map_queens = folium.Map(location=[queens_latitude, queens_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
folium.Marker([queens_latitude, queens_longitude]).add_to(map_queens)    
map_queens

Let's transform the latitude and longitude to cartesian

In [6]:
import pyproj
import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

x_queens, y_queens = lonlat_to_xy(queens_longitude, queens_latitude)
print('Queens center UTM X={}, Y={}'.format(x_queens, y_queens))
lo, la = xy_to_lonlat(x_queens, y_queens)
print('Queens center longitude={}, latitude={}'.format(lo, la))

Queens center UTM X=-5831700.800665509, Y=9841577.272716561
Queens center longitude=-73.79142141581565, latitude=40.65249269999886


Let's add information to our dataframe with the distance to de center and coordanates x,y

In [7]:
distance=[]
X_coord=[]
Y_coord=[]
for index, row in neighborhoods.iterrows():
    xs,ys= lonlat_to_xy(row['Longitude'], row['Latitude'])
    ds=calc_xy_distance(xs, ys, x_queens, y_queens)
    X_coord.append(xs)
    Y_coord.append(ys)
    distance.append(ds)

We are interesested

In [8]:
from pandas import DataFrame
DT = {'Borough': neighborhoods['Borough'], 'Neighborhood': neighborhoods['Neighborhood'],'Latitude': neighborhoods['Latitude'],
'Longitude': neighborhoods['Longitude'], 'Distance_to_center': distance,'X':X_coord,'Y':Y_coord}
#DT = {'Distance to center': distance,'X':X_coord,'Y':Y_coord}
neighborhood_total = DataFrame(DT,columns= ['Borough','Neighborhood','Latitude','Longitude','Distance_to_center', 'X', 'Y'])
neighborhood_total.head()

,Borough,Neighborhood,Latitude,Longitude,Distance_to_center,X,Y
129,Queens,Astoria,40.768509,-73.915654,25397.033121,-5.812511e+06,9.858213e+06
130,Queens,Woodside,40.746349,-73.901842,21384.337664,-5.816215e+06,9.856324e+06
131,Queens,Jackson Heights,40.751981,-73.882821,20604.314860,-5.815188e+06,9.853901e+06
132,Queens,Elmhurst,40.744049,-73.881656,19427.365586,-5.816528e+06,9.853711e+06
133,Queens,Howard Beach,40.654225,-73.838138,6051.896551,-5.831597e+06,9.847628e+06


### Foursquare

We are interesed in Latin American restaurants.

In [9]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [10]:
CLIENT_ID = 'U4LDCM2YGHXFZ0WVVL3PYH0SXJIYXRGBXZZ5L0AYSL1VV3L4' # your Foursquare ID
CLIENT_SECRET = 'BDRNWMYY2EGIIY115ZIM2E1F5532042SI2IXUDU45WQXPYZJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [11]:

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

latin_american_restaurant_categories = ['56aa371be4b08b9a8d573558','4eb1bfa43b7b52c0e1adc2e8', '58daa1558bbb0b01f18ec1f4', '4bf58dd8d48988d16b941735', '4bf58dd8d48988d107941735', '4bf58dd8d48988d1cd941735']

In [12]:
def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [13]:
restaurants = {}
latin_american_restaurants = {}
location_restaurants = []

for lat, lon in zip(neighborhood_total['Latitude'], neighborhood_total['Longitude']):
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=600, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_latin_american = is_restaurant(venue_categories, specific_filter=latin_american_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_latin_american, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_latin_american:
                    latin_american_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
print(' .', end='')
print(' done.')

 . done.


In [14]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Latin American restaurants:', len(latin_american_restaurants))
print('Percentage of Latin American restaurants: {:.2f}%'.format(len(latin_american_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1137
Total number of Latin American restaurants: 53
Percentage of Latin American restaurants: 4.66%
Average number of restaurants in neighborhood: 4.037037037037037


In [15]:
map_queens = folium.Map(location=[queens_latitude, queens_longitude], zoom_start=11)

for res in latin_american_restaurants.values():
    latl = res[2]; lonl = res[3]
    color = 'blue'
    folium.CircleMarker([latl, lonl], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_queens)
map_queens

# Methodology

In this project we will direct our efforts on detecting areas of Queen that have low Latin American restaurant density.First, we collected data of Latin American resturants thanks to Foursquare.

Next, we will calculate the area that is close to center with no Latin American restaurant.

# Analysis

In [16]:
location_restaurants_count = [len(res) for res in location_restaurants]

neighborhood_total['Restaurants_in_area'] = location_restaurants_count


neighborhood_total.head()

,Borough,Neighborhood,Latitude,Longitude,Distance_to_center,X,Y,Restaurants_in_area
129,Queens,Astoria,40.768509,-73.915654,25397.033121,-5.812511e+06,9.858213e+06,7
130,Queens,Woodside,40.746349,-73.901842,21384.337664,-5.816215e+06,9.856324e+06,16
131,Queens,Jackson Heights,40.751981,-73.882821,20604.314860,-5.815188e+06,9.853901e+06,13
132,Queens,Elmhurst,40.744049,-73.881656,19427.365586,-5.816528e+06,9.853711e+06,6
133,Queens,Howard Beach,40.654225,-73.838138,6051.896551,-5.831597e+06,9.847628e+06,8


In [17]:
distances_to_latin_american_restaurant = []

for area_x, area_y in zip(neighborhood_total['X'], neighborhood_total['Y']):
    min_distance = 10000
    for res in latin_american_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_latin_american_restaurant.append(min_distance)

neighborhood_total['Distance to Latin American restaurant'] = distances_to_latin_american_restaurant

We are going to map a Heat Map of the latin american restaurants in queens

In [18]:
latin_american_latlons = [[res[2], res[3]] for res in latin_american_restaurants.values()]
from folium import plugins
from folium.plugins import HeatMap
map_queens = folium.Map(location=[queens_latitude, queens_longitude], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(map_queens) #cartodbpositron cartodbdark_matter
HeatMap(latin_american_latlons).add_to(map_queens)
folium.Marker([queens_latitude, queens_longitude]).add_to(map_queens)
map_queens

We can see that next to the center there is none restaurant, so now we are going to find the neighborhood next to the center that has 0 restaurants and is near to the center.

In [19]:
df=neighborhood_total[neighborhood_total.Restaurants_in_area == 0]

In [20]:
df.head()

,Borough,Neighborhood,Latitude,Longitude,Distance_to_center,X,Y,Restaurants_in_area,Distance to Latin American restaurant
135,Queens,Forest Hills,40.725264,-73.844475,14129.607716,-5.819568e+06,9.848819e+06,0,720.558778
148,Queens,South Ozone Park,40.668550,-73.809865,3623.773164,-5.829051e+06,9.844049e+06,0,5947.235538
150,Queens,Whitestone,40.781291,-73.814202,22051.347177,-5.809951e+06,9.845213e+06,0,6243.070425
156,Queens,Bellerose,40.728573,-73.720128,15864.646044,-5.818487e+06,9.832798e+06,0,7524.782622
159,Queens,Briarwood,40.710935,-73.811748,10265.388297,-5.821866e+06,9.844520e+06,0,2689.546823


In [21]:
min_row=df['Distance_to_center'].min()

In [22]:
df1=df[df.Distance_to_center == min_row]
df1

,Borough,Neighborhood,Latitude,Longitude,Distance_to_center,X,Y,Restaurants_in_area,Distance to Latin American restaurant
148,Queens,South Ozone Park,40.66855,-73.809865,3623.773164,-5.829051e+06,9.844049e+06,0,5947.235538


In [28]:
latin_american_latlons = [[res[2], res[3]] for res in latin_american_restaurants.values()]
from folium import plugins
from folium.plugins import HeatMap
map_queens = folium.Map(location=[queens_latitude, queens_longitude], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(map_queens) #cartodbpositron cartodbdark_matter
HeatMap(latin_american_latlons).add_to(map_queens)

folium.Marker([40.66855, -73.809865]).add_to(map_queens)
folium.Circle([40.66855, -73.809865], radius=1000, fill=False, color='red').add_to(map_queens)

folium.Marker([queens_latitude, queens_longitude]).add_to(map_queens)
map_queens

# Results

Our analysis shows that there are 53 Latin American restaurants in Queens. From the heat map it is observed that most of these restaurants are located far from the center of Queens. To determine which is the best place for the restaurant, the data frame was filtered through neighborhoods that do not have any Latin American restaurants nearby. Of these neighborhoods the one that is closest to the city center is the South Ozaka Park neighborhood, which is marked with a circle of red on the heat map.

# Conclusion

The best neighborhood to start a Latin American restaurant is South Ozaka Park